# Worksheet 6: Hypothesis Testing

###  Lecture and Tutorial Learning Goals
After completing this week's lecture and tutorial work, you will be able to:

1. Give an example of a question you could answer with a hypothesis test.
2. Differentiate composite vs. simple hypotheses.
3. Given an inferential question, formulate null and alternative hypotheses to be used in a hypothesis test.
4. Identify the steps and components of a basic hypothesis test ("there is only one hypothesis test").
5. Write computer scripts to perform hypothesis testing via simulation, randomization and bootstrapping approaches, as well as interpret the output.
6. Describe the relationship between confidence intervals and hypothesis testing.
7. Discuss the potential limitations of this simulation approach to hypothesis testing.

In [ ]:
# Run this cell before continuing.
library(cowplot)
library(digest)
library(gridExtra)
library(infer)
library(repr)
library(tidyverse)
library(datateachr)
source("tests_worksheet_06.R")

## 1. Short Recap & Warm-Up Question

The hypothesis testing problem is very similar to the confidence intervals problem you learned in Week 4.  There is just a shift in focus. For confidence intervals, we want to find plausible values for the parameter given a sample. In hypothesis testing, we want to find  "plausible" values for a statistic given a fixed value for the parameter. For example,  given a sample average $\bar{x}=2$, confidence intervals aim to find plausible values for the populational mean $\mu$. On the other hand, hypothesis tests assume a population parameter, say $\mu=2$, and aims to check if the obtained $\bar{x}$ is "compatible" with that value.

Before we start, let us refresh our memory on confidence intervals.

**Question 1.1**
<br>{points: 1}

When calculating a confidence interval, we are looking to find plausible values for a:

A. statistic;

B. parameter;

C. observations in the sample;

D. observations in the population;

_Assign your answer to an object called `answer1.1`. Your response should be a single character surrounded by quotes._

In [ ]:
# answer1.1 <- ...

# your code here
fail() # No Answer - remove if you provide an answer

answer1.1

In [ ]:
test_1.1()

**Question 1.2**
<br>{points: 1}

Consider the population of all UBC students in a given year. We want the proportion of students that have at least one car. 

Complete the sentences below using one of the following two options:

- `A` constant
- `B` random

--------------------

Before we take a sample:

1. The elements of the sample are ...
2. The sample proportion is ...
3. The sample standard error is ...
4. The boundaries of a confidence interval are ...
5. The parameter $p$ is ...

After we take the sample:

6. The elements of the sample are ...
7. The sample proportion is ...
8. The sample standard error is ...
9. The boundaries of a confidence interval are ...
10. The parameter $p$ is ...
11. The elements of bootstrap samples are ...



Your answer should be a string containing the letters associated with the terms in the same order as the sentences they complete. For example, one potential solution is "AAABBAAABB".

_Assign your answer to an object called `answer1.2`._

In [ ]:
# answer1.2 <- ...

# your code here
fail() # No Answer - remove if you provide an answer

answer1.2

In [ ]:
test_1.2()

## 2. Introduction to Hypothesis Testing

To introduce the idea of hypothesis testing, let us consider the Hepatitis C Virus (HCV) dataset. HCV is a virus that damages the liver. The HCV dataset contains several measurements obtained from blood tests at different stages of the disease, which are, in increasing order of severity: (1) No-Fibrosis; (2) Fibrosis; and (3) Cirrhosis. Let us take a look at the dataset first:

In [ ]:
hcv_dataset <-
    read_csv("data/hcv-data-set.csv") %>% 
    mutate(category = fct_recode(category, "No-Fibrosis" = "Hepatitis")) %>% 
    filter(category != "Blood Donor") %>% 
    mutate(category = fct_drop(category))

head(hcv_dataset)

Albumin is a protein produced by the liver. Since Hepatitis C causes liver damage, one might suspect that people with hepatitis C would have lower albumin levels than healthy people. Although this is quite reasonable, is there enough evidence to support this claim? The difference in albumin levels might be so big (or so small) that we could easily answer this question with a simple plot. In other cases, however, the answer is not entirely clear. 

The medical community has established that the average level of albumin in people with a healthy liver is `44g/L`.
In the next exercise, you will start investigating the level of albumin in patients carrying HCV. 

**Question 2.1**
<br>{points: 1}

Plot the boxplots of the `albumin` level for each stage of the disease by filling in the scaffolding below. Let us also add a line to represent the level of albumin in people with a healthy liver.

_Assign your plot to an object called `boxplots`._

In [ ]:
#boxplots <- 
#    ... %>%  
#    ggplot(aes(x = ..., y = albumin)) + 
#    geom_...() + 
#    ylab("albumin g/L") +
#    ...(...) +
#    theme(text = element_text(size=25)) +
#    geom_hline(yintercept=..., color="blue")

# your code here
fail() # No Answer - remove if you provide an answer
boxplots

In [ ]:
test_2.1()

**Question 2.2**
<br>{points: 1}

Compare the boxplots of albumin levels in each group against the mean albumin level of people with healthy livers. Then, for each category, please select the statement you think is most suitable. 

Statements: 

a. The boxplot shows a sample distribution that is not compatible with a mean level of albumin of 44g/L. In other words, it would be very surprising (or unlucky!) to obtain such a sample distribution from a population with a mean level of albumin of 44g/L.

b. The boxplot shows a sample distribution that is compatible with a mean level of albumin of 44g/L. 

c. The boxplot shows some indications that the sample does not come from a population with a mean albumin level of 44g/L. However, it is hard to tell if the distinction is due to the sampling variability or a real difference in the mean albumin level.

_Assign the letter of the statement `"a"`, `"b"`, or `"c"`, for each of the groups in object `answer2.2`._

In [ ]:
answer2.2 <- NULL
#answer2.2['No-Fibrosis'] <- 
#answer2.2['Fibrosis'] <- 
#answer2.2['Cirrhosis'] <- 

# your code here
fail() # No Answer - remove if you provide an answer

answer2.2

In [ ]:
test_2.2()

Note that we are not trying to find the plausible values for the true mean of the albumin level for each category based on our sample (which would be a confidence interval). Instead, we are trying to see if the sample we have is compatible with a hypothetical scenario of interest: the categories have the same mean level of albumin as people with a healthy liver. In other words, would it be plausible to obtain the sample we got if the hypothetical scenario of interest was real? 

Hypothesis testing is like a counter-proof. We are not trying to prove that a hypothetical scenario is real. We are checking if there is enough evidence in our sample to contradict the hypothesis (i.e., our sample is "too incompatible" with such a hypothetical scenario).

But what is a hypothesis precisely? A hypothesis is a statement about the population. Some examples of hypotheses:

1. The population is normally distributed.
2. The population mean, $\mu$, is equal to a specified value $\mu_0$.
3. The population proportion, $p$, is higher than a specified value of $p_0$. 

Although a hypothesis can be more general (like Example 1 above), hypotheses frequently refer to a population parameter such as mean, proportion, or variance. A hypothesis testing consists of two competing hypotheses: (1) $H_0$, the _null hypothesis_; and (2) $H_A$ (or $H_1$), the alternative hypothesis. The null hypothesis is generally the status quo, i.e., the hypothesis that no change has happened. It is assumed that $H_0$ and $H_A$ cover all the possible scenarios. (this means that either $H_0$ or $H_A$ is true -- but we do not know which). 

**Question 2.3**
<br>{points: 1}

In HCV dataset, we are investigating if the liver damage caused by the Hepatitis C Virus will reduce the albumin level compared to people with a healthy liver. What is the _null hypothesis_ we are testing? Where $\mu$ = true mean albumin level of people with Hepatitis C. 

A. $H_0: \mu<44g/L$

B. $H_0: \mu=44g/L$

C. $H_0: \mu>44g/L$

D. $H_0: \mu\neq44g/L$


_Assign your answer to an object called `answer2.3`. Your answer should be one of `"A"`, `"B"`, `"C"`, or `"D"`._

In [ ]:
#answer2.3 <- 

# your code here
fail() # No Answer - remove if you provide an answer

answer2.3

In [ ]:
test_2.3()

**Question 2.4**
<br>{points: 1}

In HCV dataset, we are investigating if the liver damage caused by the Hepatitis C Virus will reduce the albumin level compared to people with a healthy liver. What is the _alternative hypothesis_? Where $\mu$ = true mean albumin level of people with Hepatitis C. 

A. $H_A: \mu<44g/L$

B. $H_A: \mu=44g/L$

C. $H_A: \mu>44g/L$

D. $H_A: \mu\neq44g/L$


_Assign your answer to an object called `answer2.4`. Your answer should be one of `"A"`, `"B"`, `"C"`, or `"D"`._

In [ ]:
#answer2.4 <- 

# your code here
fail() # No Answer - remove if you provide an answer

answer2.4

In [ ]:
test_2.4()

Now that you have your _null hypothesis_ and _alternative hypothesis_ defined, it is time to conduct the hypothesis test, i.e., to check if there is enough evidence in your data to say that the _null hypothesis_ is false. But first, we need to understand what is meant by "enough evidence".  

**Question 2.5**
<br>{points: 3}

For now, assume that the `hcv_dataset` contains the entire population of patients with `Fibrosis`. Your job is to do three things: 

A. Fill in the code below to get the mean albumin level of patients with `Fibrosis`. 
_Assign the result to an object called `answer2.5_A`_.

B. Is the albumin level of patients with `Fibrosis` the same, lower, or higher than the albumin level of patients with healthy liver? _Assign the string "lower", "same", or "higher", to an object called `answer2.5_B`_.

C. True or false: we can conclude with absolute certainty, just based on the mean value obtained in `Item A`, that the mean albumin level of patients with `Fibrosis` is lower than that of patients with a healthy liver. _Assign "TRUE" or "FALSE" to an object called `answer2.5_C`_ 

In [ ]:
#answer2.5_A <- 
#    hcv_dataset %>% 
#    filter(category == ...) %>% 
#    summarise(mean_albumin = ...)

#answer2.5_B <- ...
#answer2.5_C <- ...

# your code here
fail() # No Answer - remove if you provide an answer

answer2.5_A
answer2.5_B 
answer2.5_C

In [ ]:
test_2.5_A()

In [ ]:
test_2.5_B()

In [ ]:
test_2.5_C()

However, contrary to the previous question's assumption, `hcv_dataset` does not contain the entire population of patients with `Fibrosis`. We do not have access to the entire population, so we cannot calculate the parameter of interest and compare it with the hypothesized value. The decision to reject or not reject $H_0$ will be based on a sample. The first thing we need to decide is the sample statistic that we will use to test the _null hypothesis_. This statistic is known as _test statistic_. A test statistic is a point estimate/sample statistic formula used for hypothesis testing.

**Question 2.6**
<br>{points: 3}

Considering the two hypotheses defined in Questions 2.3 and 2.4, which of the statistics below is adequate to be used as the test statistic?

A. the sample median $Q_2$.

B. the sample mean $\bar{x}$.

C. the sample standard deviation: $s=\sqrt{\frac{1}{n-1}\sum_{i=1}^n\left(x_i-\bar{x}\right)^2}$

D. the constant $\mu_0 = 44$

_Assign your answer to an object called `answer2.6`. Your answer should be a single character surrounded by quotes._

In [ ]:
# answer2.6 <- ...

# your code here
fail() # No Answer - remove if you provide an answer

answer2.6

In [ ]:
test_2.6()

**Question 2.7**
<br>{points: 3}

A difficulty that arises is that the test statistic is dependent on the sample. Different samples provide different values for the test statistic. Therefore, we cannot just compare the test statistic's value with the hypothesized value of 44 g/L of albumin.

In this exercise, you will obtain the bootstrapped sampling distribution of your test statistic using 10,000 replications. 

_Assign your answer to an object called `samp_dist_mean_albumin`._

In [ ]:
set.seed(7) # Do not change this!

fibrosis <-                         
    hcv_dataset %>%                 
    filter(category == "Fibrosis") 

#samp_dist_mean_albumin <- 
#     fibrosis %>% 
#     specify(response = ...) %>% 
#     generate(type = ..., reps = ...) %>% 
#     calculate(stat = ...)


# your code here
fail() # No Answer - remove if you provide an answer

head(samp_dist_mean_albumin)

In [ ]:
test_2.7()

**Question 2.8**
<br>{points: 1}

Fill in the code below to obtain the observed test statistic.

_Assign your answer to an object named `obs_test_stat`_.

In [ ]:
#obs_test_stat <- ...(fibrosis$albumin)

# your code here
fail() # No Answer - remove if you provide an answer

obs_test_stat

In [ ]:
test_2.8()

**Question 2.9**
<br>{points: 3}

Fill in the code below to plot the bootstrap sampling distribution you obtained in Question 2.7.

_Assign your answer to an object called `samp_dist_mean_albumin_plot`._

In [ ]:
#samp_dist_mean_albumin_plot <- 
#     ... %>% 
#     ggplot() + 
#     geom_...(aes(..), bins = 15, color="white") +
#     geom_vline(xintercept = obs_test_stat, color = "red", alpha=.3, lwd=2) + 
#     xlab(...) + 
#     theme(text = element_text(size=25)) + 
#     ggtitle("Bootstrapped sampling dist.", subtitle = "Mean albumin level ") +
#     annotate("text", x = 43.6, y = 2150, label = "Observed test statistic", color="red", size=7)

# your code here
fail() # No Answer - remove if you provide an answer

samp_dist_mean_albumin_plot

In [ ]:
test_2.9()

**Question 2.10: the Null model**
<br>{points: 3}

The test statistic's sampling distribution under the _null hypothesis_ ($H_0$) is called _null model_ or _null distribution_. Up to now, you have been studying the sampling distribution of a statistic using bootstrap simulation. This time you will use bootstrap to examine how the sampling distribution of your test statistic would look like if $H_0$ were true. 

The _null hypothesis_ states that the population mean is 44 g/L, which implies that, under $H_0$, the mean of the test statistic's sampling distribution is 44 g/L. Your job here is to obtain the _null model_ by recentering the sampling distribution stored in `samp_dist_mean_albumin` to 44 g/L.

_Assign your answer to an object named `null_model`_.

In [ ]:
#null_model <- 
    #samp_dist_mean_albumin %>% 
    #mutate(...)

# your code here
fail() # No Answer - remove if you provide an answer

head(null_model)

In [ ]:
test_2.10()

**Question 2.11: the Null model**
<br>{points: 3}

Fill in the code below to plot the _null model_ you obtained in the previous question. Also, add a vertical line to the plot at the observed value of the test statistic. 

_Assign your answer to an object called `null_model_plot`_.

In [ ]:
#null_model_plot <-
#     null_model %>% 
#     ggplot() +
#     geom_...(..., bins = 15, color="white") +
#     geom_vline(xintercept = obs_test_stat, color = "red", alpha=.3, lwd=2) + 
#     xlab("Mean albumin level (g/L)") + 
#     theme(text = element_text(size=25)) + 
#     ggtitle("Simulated null distribution", subtitle = "Mean albumin level") +
#     annotate("text", x = 43.6, y = 2200, label = "Observed test statistic", color="red", size=7)

# your code here
fail() # No Answer - remove if you provide an answer

null_model_plot

In [ ]:
test_2.11()

**Question 2.12: p-value**
<br>{points: 3}

Is the observed value of the test statistic a plausible value to be obtained if $H_0$ were true? To answer this question, you will calculate the probability of getting a value more "extreme" than the observed test statistic under the null distribution. This probability is called _p-value_. 

_Assign your answer to an object called p_value_.

In [ ]:
#p_value <- mean(null_model$stat < ...)

# your code here
fail() # No Answer - remove if you provide an answer

p_value

In [ ]:
test_2.12()

**Question 2.13: Decision**
<br>{points: 3}

Based on the _p-value_ you got in the previous question, which of the following do you think better describes the situation?

A. The low value of the p-value shows that it is entirely plausible to obtain the observed test statistic if $H_0$ were true. Therefore, $H_0$ should not be rejected.

B. The low value of the p-value shows that it is quite unlikely to get the observed test statistic if $H_0$ were true, which _certainly_ shows that $H_0$ is false. Therefore, $H_0$ should be rejected. 

C. The low value of the p-value shows that it is quite unlikely to get the observed test statistic if $H_0$ were true, which _suggests_ that $H_0$ is false. Therefore, $H_0$ should be rejected. 

D. The _p-value_ is quite low, and since the _p-value_ is the probability that $H_0$ is true, we should reject $H_0$. 

_Assign your answer to an object called `answer2.13`. Your answer should be a single character surrounded by quotes._

In [ ]:
#answer2.13 <-

# your code here
fail() # No Answer - remove if you provide an answer

answer2.13

In [ ]:
test_2.13()

**Question 2.14: Types of Error**
<br>{points: 3}

There are two possible errors we could make in hypothesis testing:

1. Type I Error: happens when we wrongly reject $H_0$ (i.e., we reject $H_0$ when $H_0$ is true);
2. Type II Error: happens when we wrongly do not reject $H_0$ (i.e., we do not reject $H_0$ when $H_0$ is False);
![](imgs/errors_table.png)

Considering the decision you made in Question 2.13, which type of error are you at risk of making? 

A. Type I Error

B. Type II Error

_Assign your answer to an object called `answer2.14`._

In [ ]:
#answer2.14 <-

# your code here
fail() # No Answer - remove if you provide an answer

answer2.14

In [ ]:
test_2.14()

**Question 2.15: Significance level**
<br>{points: 3}

We know that a low _p-value_ is evidence against $H_0$. But how low must the _p-value_ be for us to decide to reject $H_0$? 

When performing hypothesis testing, we must set the so-called _significance level_. The significance level, $\alpha$, is the probability of _Type I Error_. We will reject $H_0$ if the p-value is smaller than the significance level we chose. Typical values of $\alpha$ are $10\%, 5\%$, and $1\%$. It is important that you specify the $\alpha$ level before conducting the hypothesis testing and obtaining the p-value. 

For this question, you must select all the significance level below for which we would reject $H_0$:

A. $10\%$

B. $5\%$

C. $1\%$

D. $0.1\%$

E. None of the above.

Your answer should be a string containing the letters associated with the items you selected in the same order as the items appear. For example, if you want to select `B` and `D`, you should use `"BD"`,  not `"DB"`. 

_Assign your answer to an object called `answer2.15`._

In [ ]:
#answer2.15 <- 

# your code here
fail() # No Answer - remove if you provide an answer

answer2.15

In [ ]:
test_2.15()

**Question 2.16: hypothesis testing with the `infer` package**
<br>{points: 3}

The `infer` package has a similar [workflow to conduct hypothesis tests](https://moderndive.com/9-hypothesis-testing.html#ht-infer) to the one you have been using for confidence intervals. 

In this question, you will conduct the same hypothesis test you just did manually, but this time you are going to use the `infer` package. Fill in the code below to generate 10,000 bootstrap samples from the null model using the `infer` workflow. Then, try comparing the first ten rows of the model you manually generated `null_model` with the first ten rows you obtained using `infer`

_Assign your answer to an object called `null_model_infer`_. 

In [ ]:
set.seed(7) # Do not change this.

#null_model_infer <- 
#    fibrosis %>% 
#    specify(...) %>% 
#    hypothesise(...) %>% 
#    generate(...) %>% 
#    calculate(...)

# your code here
fail() # No Answer - remove if you provide an answer

head(null_model_infer)

In [ ]:
test_2.16()

**Question 2.17: Visualizing the result of the hypothesis test `infer`**
<br>{points: 3}


The `infer` package also makes it easy to visualize the result of your hypothesis test with `visualize` and `shade_p_value` functions (see [Section 9.3.1 of Modern Dive](https://moderndive.com/9-hypothesis-testing.html#ht-infer)).

Fill in the code below to visualize the result of your hypothesis test. 

_Assign the answer to an object named `null_model_vis_infer`_

In [ ]:
#null_model_vis_infer <-
#    null_model_infer %>% 
#    visualize(...) + 
#    shade_p_value(obs_stat = ..., direction = ...) +
#    xlab("Mean albumin level (g/L)") + 
#    theme(text = element_text(size=20))

# your code here
fail() # No Answer - remove if you provide an answer

null_model_vis_infer

In [ ]:
test_2.17()

**Question 2.18: Getting the p-value with `infer`**
<br>{points: 1}

To get the p-value with the `infer` package, we use the `get_p_value` function. Obtain the p-value using the `null_model_infer`. 

_Assign your answer to p_value_infer_

In [ ]:
#p_value_infer <- 
#    null_model_infer %>% 
#    get_p_value(obs_stat = ..., direction = ...)

# your code here
fail() # No Answer - remove if you provide an answer

p_value_infer

In [ ]:
test_2.18()

## 3. Hypothesis testing with two populations

In the previous section, the hypothesis involved only one parameter from one population: the mean level of albumin in patients with `Fibrosis`. 

In this section, you will work with two populations, and your hypothesis will involve a parameter from each population. Although it seems more complicated, fear not, the process is pretty much the same: 

1. Specify the variable of interest (`specify()`);
2. Define your hypotheses (`hypothesise()`);
3. Simulate the observations (`generate()`);
4. Generate values from the null model (`caculate()`);
5. See how the observed statistic compares with the sampling distribution by checking the p-value.

In the following sequence of exercises, we will try to answer the following question: 

> Is the diameter of the trees in Kitsilano bigger than in Kerrisdale?

**Question 3.1: What are we comparing exactly?**
<br>{points: 2}

Different trees will have different diameters. We need a summary quantity to summarise each population: (1) all the trees in Kitsilano; and (2) all the trees in Kerrisdale. Which of the following quantities are appropriate to help answer the question (select all that apply):

A. Population mean;

B. Population variance;

C. Population median;

D. Population mode; 

_Assign your answer to an object called `answer3.1`. Your response should be a sequence of characters, e.g., "ABCD"._

In [ ]:
#answer3.1 <-

# your code here
fail() # No Answer - remove if you provide an answer

answer3.1

In [ ]:
test_3.1()

**Question 3.2: Setting the hypothesis**
<br>{points: 2}

If we were to use the median, say $m_1$ is the median diameter of the trees in Kitsilano and $m_2$ the median diameter of the trees in Kerrisdale, what are the appropriate hypotheses?

A. $H_0: m_1-m_2 = 0$ vs $H_1: m_1-m_2 < 0$

B. $H_0: m_1-m_2 = 0$ vs $H_1: m_1-m_2 > 0$

C. $H_0: m_1-m_2 > 0$ vs $H_1: m_1-m_2 < 0$

D. $H_0: m_1-m_2 < 0$ vs $H_1: m_1-m_2 = 0$

_Assign your answer to an object called `answer3.2`._

In [ ]:
#answer3.2 <-

# your code here
fail() # No Answer - remove if you provide an answer

answer3.2

In [ ]:
test_3.2()

**Question 3.3: The truth!**
<br>{points: 3}

Let us assume that the entire population of trees in Kitsilano and Kerrisdale is stored in the `trees_pop` variable. Let's find out the truth! 
What is the median diameter of each population of trees? Fill in the code below to find out.

_Assign your answer to an object called `answer3.3`._

In [ ]:
# Run this code before continuing
trees_pop <-
    vancouver_trees %>% 
    filter(neighbourhood_name %in% c("KITSILANO", "KERRISDALE")) %>% 
    select(neighbourhood_name, diameter)

head(trees_pop)

In [ ]:
#answer3.3 <- 
#    trees_pop %>% 
#    ...(neighbourhood_name) %>% 
#    ...(median = ...)

# your code here
fail() # No Answer - remove if you provide an answer

answer3.3

In [ ]:
test_3.3()

**Question 3.4: Let's take a sample**
<br>{points: 3}

A sample of 31 trees from Kitsilano and 35 trees from Kerrisdale was taken and stored in the object `sample_trees`. Calculate the observed test statistic as the difference in the median diameter between KITSILANO & KERRISDALE

_Assign your answer to an object called `obs_med_diam_diff`_

In [ ]:
set.seed(12) # Do not change this

# Run this cell before continuing.
sample_trees <- 
    trees_pop %>% 
    filter(neighbourhood_name=="KERRISDALE") %>% 
    sample_n(size=35) %>% 
    bind_rows(
        trees_pop %>% 
            filter(neighbourhood_name=="KITSILANO") %>% 
            sample_n(size=31))

head(sample_trees)

In [ ]:
# obs_med_diam_diff <- 
#     sample_trees %>% 
#     filter(neighbourhood_name %in% c(..., ...)) %>%
#     group_by(...) %>% 
#     summarise(median = ...) %>%
#     pivot_wider(names_from = neighbourhood_name, values_from = median) %>%
#     transmute(diff = ...) %>%
#     pull(...)

# your code here
fail() # No Answer - remove if you provide an answer

obs_med_diam_diff

In [ ]:
test_3.4()

**Question 3.5: Simulating from the null distribution**
<br>{points: 3}

Fill in the code below to generate 5000 samples from the null distribution.

_Assign your answer to an object called `null_model_trees`_

In [ ]:
set.seed(50)

#null_model_trees <- 
#    sample_trees %>% 
#    specify(formula = ... ~ ...) %>% 
#    hypothesize(null = ...) %>% 
#    ...(reps = 5000, type = "permute") %>% 
#    ...(stat="diff in medians", order = c("KITSILANO", "KERRISDALE"))

# your code here
fail() # No Answer - remove if you provide an answer

head(null_model_trees)

In [ ]:
test_3.5()

**Question 3.6**
<br>{points: 3}

Fill in the code below to plot the result of the hypothesis test. 

_Assign your answer to an object called `trees_result_plot`_.

In [ ]:
#trees_result_plot <- 
#    null_model_trees %>%
#    visualize() + 
#    shade_p_value(obs_stat = ..., direction = ...)

# your code here
fail() # No Answer - remove if you provide an answer

trees_result_plot

In [ ]:
test_3.6()

**Question 3.7**
<br>{points: 3}

Obtain the p-value from `null_model_trees`.

_Assign your answer to an object called `answer3.7`_.

In [ ]:
#answer3.7 <- 

# your code here
fail() # No Answer - remove if you provide an answer

answer3.7

In [ ]:
test_3.7()

**Question 3.8**
<br>{points: 3}

What decision should be made at 10% significance level?

A. Reject $H_0$ and commit the Type I Error;

B. Reject $H_0$ and commit the Type 2 Error;

C. Correctly reject $H_0$;

D. Not reject $H_0$ and commit the Type 1 Error;

E. Not reject $H_0$ and commit the Type 2 Error;

F. Correctly not reject $H_0$;

_Assign your answer to an object called `answer3.5`_

In [ ]:
#answer3.8 <- 

# your code here
fail() # No Answer - remove if you provide an answer

answer3.8

In [ ]:
test_3.8()